In [1]:
!python --version

Python 3.7.7


In [2]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
# !pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets

In [3]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [7]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [8]:
ws = WS('./data/',disable_cuda=False)
pos = POS('./data/',disable_cuda=False)

In [9]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [10]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [11]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [12]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
ind = 0
for sent in train_title_cuts.values():
    for word,_ in sent:
        if word not in word2index:
            word2index[word]=ind
            index2word[ind]=word
            ind+=1   
# END YOUR CODE

In [13]:
word2index['溫暖']

1520

In [14]:
index2word[1520]

'溫暖'

In [15]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for ind in [word2index[w] for w,i in pairs]:
        vector[ind]+=1
    # END YOUR CODE
    return vector

In [16]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [17]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['精准', '嘉年華', '2.0', '小詹皇', '騰訊', '伊亞', '許昕', '安迪爾森', '格力', '王者榮', '四阿哥', '約翰塞納', '陶華碧', '重渡溝', '威少', '猛', '阿珂', '維亞利', '馬雲', '璞麗', '黃希揚', '特里', '馬化騰', '詹黑皮爾斯', '小智', '華創', '海王星', '韓長賦', '科斯塔', '梵', '阿里', '樸泰夏', '奧拉朱旺遠', '余小C', '賈躍亭', '老詹', '李秋平', '密子君', '賞月夜', '杜蕾斯', '聯發科', '喬丹', 'WWE', '溫格', '羅', '哈登', '塔特姆', '詹皇', '朱婷', '后羿', '曼薩諾', '星巴克', '斯威', '董明珠', '圍甲', '張大仙', '切爾西', '聯盟里', '建聯', '塔克', '世界杯', '李大霄', '丘誠', '維韋杜詹霍', '埃爾克森', '火箭隊', '吳敬璉', '網傳', '姆巴佩', '微博', '美年達', '庫里', '殺里', '盧克', '斐濟', '馬卡報', '扎堆', '拉卡拉', '萊萬特', '拜仁續', '安德森', '曼聯', '卡納瓦羅', '陰陽師', '惠若琪', '單薇恩', '卡拉黑', '意大利杯', '中超', '沃爾瑪', '安徒恩比盧克', '京新', '景泰石', '建成', '穆謝奎', '凱塔', '哈希', '張掖篇', '克洛普', '旭旭']
VC : ['前瞻', '虐爆', '欺騙', '藏', '蒙上', '參加', '產出', '春栽', '撒', '詮釋', '換', '進口', '發', '約', '拍賣', '贊取', '創', '捨棄', '播', '進出口', '攔', '問倒', '焊', '種養', '伴隨', '助推', '送走', '丟棄', '對待', '借鑒', '取消', '影響', '運用', '開掛', '變更', '禁', '產', '治療', '作', '編程', '打不過', '辦', '打理', '宣一', '要', '到', '上', '實施', '底背離', '收到', '帶', '冠絕', '碰到', '解說', '挑選出', '收

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [18]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'COLONCATEGORY','WHITESPACE','Neu','COMMACATEGORY','Nes','QUESTIONCATEGORY','EXCLAMATIONCATEGORY'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [19]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    similarity=np.dot(bow1,bow2)/(np.linalg.norm(bow1, ord=2)*np.linalg.norm(bow2, ord=2))
    # END YOUR CODE
    return similarity

In [20]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [21]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [22]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [23]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [24]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [25]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [26]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 63, '遊戲': 8, '旅遊': 6, '農業': 6, '財經': 5})
predict 財經 :  Counter({'財經': 62, '科技': 15, '農業': 9, '體育': 8, '旅遊': 8, '遊戲': 5})
predict 科技 :  Counter({'科技': 57, '農業': 18, '財經': 16, '旅遊': 13, '體育': 12, '遊戲': 8})
predict 旅遊 :  Counter({'旅遊': 54, '農業': 12, '財經': 7, '遊戲': 7, '科技': 5, '體育': 2})
predict 農業 :  Counter({'農業': 53, '科技': 13, '體育': 9, '旅遊': 8, '財經': 6, '遊戲': 5})
predict 遊戲 :  Counter({'遊戲': 67, '旅遊': 10, '科技': 9, '體育': 5, '財經': 4, '農業': 2})
